## Cash Flow 계산 Test

In [121]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

import pandas as pd

pd.options.display.float_format = '{:.2f}'.format

In [122]:
dateOfIssue = '20210101'
dateOfMaturity = '20250101'

In [123]:
dateOfIssue = datetime.strptime(dateOfIssue, '%Y%m%d') # 문자열 -> datetime 변환
dateOfMaturity = datetime.strptime(dateOfMaturity, '%Y%m%d') # 문자열 -> datetime 변환

print('채권 발행일', dateOfIssue)
print('채권 만기일', dateOfMaturity)

채권 발행일 2021-01-01 00:00:00
채권 만기일 2025-01-01 00:00:00


In [124]:
period = 3 # 이자지급주기 (월)

In [134]:
cfDay = dateOfIssue
for _ in range(20):
    cfDay += relativedelta(months=period)
    print(cfDay)
    
    if cfDay == dateOfMaturity:
        print('Done!')
        break

2021-04-01 00:00:00
2021-07-01 00:00:00
2021-10-01 00:00:00
2022-01-01 00:00:00
2022-04-01 00:00:00
2022-07-01 00:00:00
2022-10-01 00:00:00
2023-01-01 00:00:00
2023-04-01 00:00:00
2023-07-01 00:00:00
2023-10-01 00:00:00
2024-01-01 00:00:00
2024-04-01 00:00:00
2024-07-01 00:00:00
2024-10-01 00:00:00
2025-01-01 00:00:00
Done!


In [135]:
cfEnd = dateOfIssue
cRate=0.03
faceValue = 10_000_000_000

cashFlow = list()
while True:
    if cfEnd == dateOfMaturity:
        print('Done!')
        break
        
        
    cfStart = cfEnd
    cfEnd = cfEnd+relativedelta(months = period)
    
    cfDiffDays = cfEnd - cfStart
    cfAmount = faceValue*cRate
    
    cashFlow.append([cfStart, cfEnd, cfDiffDays, 1, cfAmount, cfAmount])

Done!


In [136]:
df = pd.DataFrame(cashFlow, columns=['CF_StartDate', 'CF_EndDate', 'CF_Period', 'Weighted_R','CF_E_Amount', 'CF_R_Amount'])
df

,CF_StartDate,CF_EndDate,CF_Period,Weighted_R,CF_E_Amount,CF_R_Amount
0,2021-01-01,2021-04-01,90 days,1,300000000.00,300000000.00
1,2021-04-01,2021-07-01,91 days,1,300000000.00,300000000.00
2,2021-07-01,2021-10-01,92 days,1,300000000.00,300000000.00
3,2021-10-01,2022-01-01,92 days,1,300000000.00,300000000.00
4,2022-01-01,2022-04-01,90 days,1,300000000.00,300000000.00
5,2022-04-01,2022-07-01,91 days,1,300000000.00,300000000.00
6,2022-07-01,2022-10-01,92 days,1,300000000.00,300000000.00
7,2022-10-01,2023-01-01,92 days,1,300000000.00,300000000.00
8,2023-01-01,2023-04-01,90 days,1,300000000.00,300000000.00
9,2023-04-01,2023-07-01,91 days,1,300000000.00,300000000.00


In [137]:
dayOfPurchase = datetime.strptime('20220501','%Y%m%d')
dayOfPurchase

datetime.datetime(2022, 5, 1, 0, 0)

In [138]:
idx=0
newCf=list()
while True:
    if idx==len(df):
        break
        
    if dayOfPurchase > df.loc[idx,'CF_EndDate']:
        df.loc[idx,'CF_R_Amount'] = 0
    else:
        diffDay1 = dayOfPurchase - df.loc[idx,'CF_StartDate']
        newCf.extend([
              df.loc[idx,'CF_StartDate']
            , dayOfPurchase
            , diffDay1
            , diffDay1/df.loc[idx,'CF_Period']
            , df.loc[idx,'CF_E_Amount']
            , df.loc[idx,'CF_E_Amount']*(diffDay1/df.loc[idx,'CF_Period'])
        ])
        
        df.loc[idx,'CF_StartDate']=dayOfPurchase
        
        diffDay2 = df.loc[idx,'CF_EndDate'] - dayOfPurchase
        df.loc[idx,'Weighted_R']=diffDay2/df.loc[idx,'CF_Period']
        
        df.loc[idx,'CF_R_Amount']=df.loc[idx,'CF_E_Amount']*df.loc[idx,'Weighted_R']
        df.loc[idx,'CF_Period']=diffDay2
        
        break
        
    idx+=1

In [139]:
newCf

[Timestamp('2022-04-01 00:00:00'),
 datetime.datetime(2022, 5, 1, 0, 0),
 Timedelta('30 days 00:00:00'),
 0.32967032967032966,
 300000000.0,
 98901098.90109889]

In [140]:
df.loc[len(df)]=newCf

df.sort_values(by='CF_StartDate', ignore_index=True, inplace=True)
df

,CF_StartDate,CF_EndDate,CF_Period,Weighted_R,CF_E_Amount,CF_R_Amount
0,2021-01-01,2021-04-01,90 days,1.00,300000000.00,0.00
1,2021-04-01,2021-07-01,91 days,1.00,300000000.00,0.00
2,2021-07-01,2021-10-01,92 days,1.00,300000000.00,0.00
3,2021-10-01,2022-01-01,92 days,1.00,300000000.00,0.00
4,2022-01-01,2022-04-01,90 days,1.00,300000000.00,0.00
5,2022-04-01,2022-05-01,30 days,0.33,300000000.00,98901098.90
6,2022-05-01,2022-07-01,61 days,0.67,300000000.00,201098901.10
7,2022-07-01,2022-10-01,92 days,1.00,300000000.00,300000000.00
8,2022-10-01,2023-01-01,92 days,1.00,300000000.00,300000000.00
9,2023-01-01,2023-04-01,90 days,1.00,300000000.00,300000000.00
